## Example Use of David's Finance Module, dfinance
#### See it in action!
This notebook shows an example that implements my custom finance package to backtest an (ineffective) algorithmic trading strategy.

In [1]:
import pandas as pd
import yfinance as yf
import numpy as np
import pickle

import dfinance as dfin
from dfinance.trading_strategies import SMA_Crossover
from dfinance.trading_strategies import PaperBracketOrder
from dfinance.trading_strategies import BTBracketOrder
from dfinance.trading_strategies import BT_EMA_Crossover
from dfinance.trading_strategies import RH_EMA_Crossover

import talib
from talib.abstract import *

from robin_stocks import *
import robin_stocks.robinhood as r
import pyotp

import os
from dotenv import load_dotenv
load_dotenv()

%load_ext autoreload
%autoreload 2

### Backtest a strategy that buys based on model predictions

In [3]:
my_port = dfin.Portfolio()
my_back = dfin.Backtest()

In [135]:
# Get cash value - can also set this
my_port.cashvalue = 100

In [136]:
# load data from Yahoo!
days_of_data = 110

data = yf.download(tickers='HOG', period=f'{days_of_data}d', progress=False)
data = data.head(days_of_data - 10)

In [137]:
my_back.process_historical_data(df=data,
                                my_port=my_port,
                                my_strategy=BTBracketOrder.strategy,
                                ticker='HOG',
                                share_cnt=1,
                                profit_pct=1.03,
                                stop_pct=0.98,
                                limit_pct=0.97,
                                pkl_filename='lgbm_model.pkl')

Buying 1 shares of HOG for 36.88999938964844
Selling 1 shares of HOG for 36.70000076293945
Buying 1 shares of HOG for 37.08000183105469
Selling 1 shares of HOG for 37.08000183105469
Buying 1 shares of HOG for 37.38999938964844
Selling 1 shares of HOG for 37.70000076293945
Buying 1 shares of HOG for 36.560001373291016
Selling 1 shares of HOG for 36.220001220703125
Buying 1 shares of HOG for 36.66999816894531
Selling 1 shares of HOG for 36.5099983215332
Buying 1 shares of HOG for 35.310001373291016
Selling 1 shares of HOG for 36.29999923706055
Buying 1 shares of HOG for 36.29999923706055
Selling 1 shares of HOG for 36.93000030517578
Buying 1 shares of HOG for 36.58000183105469
Selling 1 shares of HOG for 36.04999923706055
Buying 1 shares of HOG for 36.040000915527344
Selling 1 shares of HOG for 37.290000915527344
Buying 1 shares of HOG for 36.630001068115234
Selling 1 shares of HOG for 36.5099983215332
Buying 1 shares of HOG for 36.20000076293945
Selling 1 shares of HOG for 36.1800003051

In [138]:
my_port.cashvalue

105.70999908447266

In [139]:
my_port.ledger()

,Ticker,Shares,Share Price,Dollar Amount,Buy/Sell,Transaction Datetime
0,HOG,1,36.889999,36.889999,buy,2021-09-08
1,HOG,-1,36.700001,-36.700001,sell,2021-09-09
2,HOG,1,37.080002,37.080002,buy,2021-09-10
3,HOG,-1,37.080002,-37.080002,sell,2021-09-13
4,HOG,1,37.389999,37.389999,buy,2021-09-14
...,...,...,...,...,...,...
71,HOG,-1,37.560001,-37.560001,sell,2022-01-03
72,HOG,1,38.080002,38.080002,buy,2022-01-04
73,HOG,-1,38.320000,-38.320000,sell,2022-01-05
74,HOG,1,37.919998,37.919998,buy,2022-01-06


---

### Paper trading with Alpaca
Only works when the market is open.

In [8]:
my_alpaca_port = dfin.AlpacaPortfolio()
my_back = dfin.Backtest()

In [9]:
prefill_rows = yf.download(tickers='HOG', period='100d', progress=False)

In [10]:
my_back.process_live_data(prefill_rows=prefill_rows,
                          my_alpaca_port=my_alpaca_port,
                          my_strategy=PaperBracketOrder.strategy,
                          ticker='HOG',
                          share_cnt=3,
                          profit_pct=1.05,
                          stop_pct=0.95,
                          limit_pct=0.94,
                          pkl_filename='lgbm_model.pkl')

Market is closed. No action will be taken.


In [11]:
# must call refresh() before getting status or positions
# refresh() reloads the value of all attributes with current values
my_alpaca_port.refresh()

my_alpaca_port.get_account_status()

,Cash,Equity,Last Equity,Equity Change,Equity Change %
0,99999.11,99999.11,99999.11,0.0,0.0


In [12]:
my_alpaca_port.current_positions()

,Ticker,Shares Held,Cost Basis,Current Price,Unrealized Gain/Loss,Unrealized Gain/Loss %


---

### Backtest SMA Crossover strategy

In [2]:
# Initialize backtest object
# Initialize portfolio object
my_back = dfin.Backtest()
my_port = dfin.Portfolio()

In [3]:
# Get cash value - can also set this
my_port.cashvalue

100000.0

In [133]:
# load data from Yahoo!
data = yf.download(tickers='AAPL', period='90d', progress=False)
data.to_csv('aapl_daily_data.csv')
historical_df = pd.read_csv('aapl_daily_data.csv', index_col='Date')

In [16]:
# Process the data from the historical dataframe
my_back.process_historical_data(historical_df,
                                my_port,
                                SMA_Crossover.strategy,
                                sma_short=10,
                                sma_long=15,
                                share_num=10,
                                ticker='AAPL')

2021-10-05 | Short: 142.05 | Long: 142.92 | Short < Long
2021-10-06 | Short: 141.51 | Long: 142.39 | Short < Long
2021-10-07 | Short: 141.22 | Long: 142.09 | Short < Long
2021-10-08 | Short: 140.92 | Long: 141.88 | Short < Long
2021-10-11 | Short: 140.72 | Long: 141.91 | Short < Long
2021-10-12 | Short: 140.65 | Long: 141.80 | Short < Long
2021-10-13 | Short: 140.37 | Long: 141.50 | Short < Long
2021-10-14 | Short: 140.39 | Long: 141.22 | Short < Long
2021-10-15 | Short: 140.83 | Long: 141.09 | Short < Long
2021-10-18 | Short: 141.32 | Long: 141.04 | Short > Long

Buying 10 shares of AAPL for 1431.6000366210938

2021-10-19 | Short: 142.04 | Long: 141.37 | Short > Long
2021-10-20 | Short: 143.02 | Long: 141.77 | Short > Long
2021-10-21 | Short: 143.53 | Long: 142.21 | Short > Long
2021-10-22 | Short: 144.14 | Long: 142.85 | Short > Long
2021-10-25 | Short: 144.72 | Long: 143.47 | Short > Long
2021-10-26 | Short: 145.52 | Long: 144.11 | Short > Long
2021-10-27 | Short: 146.45 | Long: 144

In [17]:
my_port.cashvalue

100329.3000793457

In [18]:
my_port.ledger()

,Ticker,Shares,Share Price,Dollar Amount,Buy/Sell,Transaction Datetime
0,AAPL,10,143.160004,1431.600037,buy,2021-10-18
1,AAPL,-10,178.529999,-1785.299988,sell,2021-12-28
2,AAPL,10,177.259995,1772.599945,buy,2021-12-31
3,AAPL,-10,174.820007,-1748.200073,sell,2022-01-12


### Demonstrate other functions of the Porfolio class

In [19]:
my_port.current_positions()

,Ticker,Shares Held,Current Price,Current Value of Shares


In [20]:
# Buy one share of GME for $100 - what a great price!
my_port.buy_stock('GME', 3, 100, '2021-01-01 00:00:00')

In [21]:
my_port.save_transactions('my_port.csv')

In [22]:
my_port.load_transactions('my_port.csv')
my_port.current_positions()

,Ticker,Shares Held,Current Price,Current Value of Shares
1,GME,3,106.0,318.0
2,Totals,3,N/A,318.0


***

## EMA BACKTEST

In [178]:
# Initialize backtest object
# Initialize portfolio object
my_back = dfin.Backtest()
my_port = dfin.Portfolio()

In [179]:
# Get cash value - can also set this
my_port.cashvalue


100000.0

In [180]:
# load data from Yahoo!
# Note, the first 500 days are just used to converge the EMAs
# Only the days after 500 are part of the backtest
data = yf.download(tickers='AAPL', period='700d', progress=False)

# Process the data from the historical dataframe
my_back.process_historical_data(data,
                                my_port,
                                BT_EMA_Crossover.strategy,
                                ema_short=15,
                                ema_long=55,
                                share_num=10,
                                ticker='AAPL',
                                close_positions_eob=True)

2021-04-19 00:00:00 | Short: 128.63 | Long: 125.63 | Short > Long

Buying 10 shares of AAPL for 1333.3999633789062

2021-04-20 00:00:00 | Short: 129.02 | Long: 125.85 | Short > Long
2021-04-21 00:00:00 | Short: 129.31 | Long: 126.05 | Short > Long
2021-04-22 00:00:00 | Short: 129.57 | Long: 126.24 | Short > Long
2021-04-23 00:00:00 | Short: 129.90 | Long: 126.45 | Short > Long
2021-04-26 00:00:00 | Short: 130.35 | Long: 126.70 | Short > Long
2021-04-27 00:00:00 | Short: 130.82 | Long: 126.97 | Short > Long
2021-04-28 00:00:00 | Short: 131.10 | Long: 127.19 | Short > Long
2021-04-29 00:00:00 | Short: 131.27 | Long: 127.37 | Short > Long
2021-04-30 00:00:00 | Short: 131.25 | Long: 127.51 | Short > Long
2021-05-03 00:00:00 | Short: 131.32 | Long: 127.66 | Short > Long
2021-05-04 00:00:00 | Short: 130.74 | Long: 127.63 | Short > Long
2021-05-05 00:00:00 | Short: 130.40 | Long: 127.64 | Short > Long
2021-05-06 00:00:00 | Short: 129.99 | Long: 127.62 | Short > Long
2021-05-07 00:00:00 | Shor

In [181]:
my_port.cashvalue

100269.3000793457

In [182]:
my_port.ledger()

,Ticker,Shares,Share Price,Dollar Amount,Buy/Sell,Transaction Datetime
0,AAPL,10,133.339996,1333.399963,buy,2021-04-19
1,AAPL,-10,125.169998,-1251.699982,sell,2021-05-17
2,AAPL,10,128.460007,1284.600067,buy,2021-06-16
3,AAPL,-10,139.110001,-1391.100006,sell,2021-10-01
4,AAPL,10,147.869995,1478.699951,buy,2021-10-21
5,AAPL,-10,172.320007,-1723.200073,sell,2022-02-01


In [183]:
my_port.current_positions()

,Ticker,Shares Held,Current Price,Current Value of Shares


***